### Questions to be answered 

- What is Assistant Agent? 

    - Assistant agent, designed to solve a task with LLM.

    - The default system message is designed to solve a task with LLM,
    including suggesting python code blocks and debugging.

    - `human_input_mode` is default to "NEVER"

    - `code_execution_config` is default to False.

    - Code Execution is `disabled` by default, and expects the user to execute the code.

    - **Not Same** as GPT Assistant Agent... 

- How it is different from User Proxy Agent? 

    - UserProxyAgent is configured with `human_input_mode` to ALWAYS
    and `llm_config` to False. 
    
    - By default, the agent `will prompt for human input` every time a message is received.
    
    - Code execution is enabled by default. LLM-based auto reply is disabled by default.

- What problems it solves?
    - The prompt is designed to get the tasks solved by the GPT models inside 
    OpenAI or your own servers.  
- Use Cases... 

In [ ]:
import autogen
import os
from dotenv import load_dotenv

config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../.env',
    model_api_key_map={
        "gpt-3.5-turbo":"OPENAI_API_KEY"
    },
    filter_dict={
        "model":"gpt-3.5-turbo"
    }
)
# config_list

In [ ]:
asst_id = os.environ.get('ASST_ID')
asst_id == ''
# no need for Openai assistant in this notebook. As only the regular completions are used

In [ ]:
llm_config = {
    "config_list": config_list,
}
llm_config

In [ ]:
planner = autogen.AssistantAgent(
    name="planner",
    llm_config=llm_config,
    # the default system message of the AssistantAgent is overwritten here
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step that can be implemented by writing code. For example, browsing the web can be implemented by writing code that reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix."
)

In [ ]:
planner_user = autogen.UserProxyAgent(
    name="planner_user",
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
)

def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    # return the last message received from the planner
    return planner_user.last_message()["content"]


#### How is the plan recieved? Via Function call. A function call through OpenAI API

In [ ]:
output_check = planner_user.initiate_chat(recipient=planner, message="How to start learning about Python?")

In [ ]:
print(planner_user.last_message()['content'])

In [ ]:
proxy_with_one_autoreply = autogen.UserProxyAgent(
    name='one_auto_reply',
    max_consecutive_auto_reply=1,
    human_input_mode='NEVER',
)
proxy_with_one_autoreply.initiate_chat(recipient=planner, message="Connect with the AWS instance running at 100.100.10.18")

In [ ]:
proxy_with_one_human_reply = autogen.UserProxyAgent(
    name="one_human_reply",
    max_consecutive_auto_reply=1,  #this is not required
    human_input_mode='ALWAYS',
)
proxy_with_one_human_reply.initiate_chat(recipient=planner, message="Connecting via ssh to a machine on lan")

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for feedback when it receives a "TERMINATE" signal from the assistant agent. We set the `functions` in `AssistantAgent` and `function_map` in `UserProxyAgent` to use the created `ask_planner` function.

In [ ]:
assistant_config = {
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
        "functions": [
            {
                "name": "ask_planner",
                "description": "ask planner to: 1. get a plan for finishing a task, 2. verify the execution result of the plan and potentially suggest new plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask planner. Make sure the question include enough context, such as the code and the execution result. The planner does not know the conversation between you and the user, unless you share the conversation with the planner.",
                        },
                    },
                    "required": ["message"],
                },
            },
        ],
        "model":"gpt-3.5-turbo"
    }
assistant_config

In [ ]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=assistant_config
)

In [ ]:
print(assistant.system_message)

In [ ]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=3,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "planning"},
    function_map={"ask_planner": ask_planner},
)
# This cell creates a agent, that accepts human input only after the model 
# has sent a message containing TERMINATE. Or allows at max 3 auto-replies
# This agent doesn't connect with OpenAI Models for inference

## Perform a task

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal at the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

##### In this case planner and planner_user are not called

In [ ]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Suggest a formula to find the temperature inside the sun of our Solar system""",
)

##### In this case we see the call to planner and planner_user

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="""How will you plan to harness all the energy given by Earth's Sun""",
)

In [ ]:
print(planner_user.last_message()['content'])

#### Where the UserProxyAgent / AssistantAgent generate replies
`line 854 on conversable_agent.py`
```
def generate_reply(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[Agent] = None,
        exclude: Optional[List[Callable]] = None,
    ) -> Union[str, Dict, None]:
        """Reply based on the conversation history and the sender.

        Either messages or sender must be provided.
        Register a reply_func with `None` as one trigger for it to be activated when `messages` is non-empty and `sender` is `None`.
        Use registered auto reply functions to generate replies.
        By default, the following functions are checked in order:
        1. check_termination_and_human_reply
        2. generate_function_call_reply
        3. generate_code_execution_reply
        4. generate_oai_reply
        Every function returns a tuple (final, reply).
        When a function returns final=False, the next function will be checked.
        So by default, termination and human reply will be checked first.
        If not terminating and human reply is skipped, execute function or code and return the result.
        AI replies are generated only when no code execution is performed.
```